In [77]:
# %load ../src/pipeline_classes.py
import pandas as pd 
import numpy as np 
from sklearn.base import BaseEstimator, TransformerMixin

class Featurizer(BaseEstimator, TransformerMixin):
    """Transform incoming df to fit into model"""
   
    def __init__(self, cols=None):
        """INPUT: an optional cols list of columns to select"""
        if cols==None:
            self.cols = ['date', 'temp', 'precipitation', 'overcast', 'poor_visibility', 'windy']
        else:
            self.cols = cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """tranform incoming training or test"""
        df = X.copy()
        df.index = df.date
        date_column = pd.Series(df.index)
        month_day_of_week = pd.DataFrame({"year": date_column.dt.year,
                                        "month": date_column.dt.month, 
                                        "day": date_column.dt.day,
                                        "dayofweek": date_column.dt.dayofweek})
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 0] = 'Monday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 1] = 'Tuesday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 2] = 'Wednesday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 3] = 'Thursday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 4] = 'Friday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 5] = 'Saturday'
        month_day_of_week.dayofweek[month_day_of_week.dayofweek == 6] = 'Sunday'
        month_day_of_week = pd.get_dummies(month_day_of_week)
        month_day_of_week.index = df.index
        features = pd.concat([df, month_day_of_week], axis=1)
        features.drop(['date'], axis=1, inplace=True)
        return features

In [10]:
# %load ../src/weather_data_clean.py
import pandas as pd

def clean_weather_data(filename):
    """Take ASOS weather data file for Stampede pass and clean it ready for input to model.
    Input: txt file
    Output: pandas dataframe
    """
    data = pd.read_csv(filename)

    # Rename two of the columns
    data.rename(columns={'valid':'date', 'tmpf':'temp'}, inplace=True)

    # Remove the few rows that have a null value for temp
    data = data[~data.temp.eq('M')]

    # Remove spaces from column names
    data.rename(columns=lambda x: x.replace(' ', ''), inplace=True)

    # Only use the standard hourly weather reading at 56 mins past each hour
    mask = data['date'].apply(lambda x: x[-2:] == '56')
    data = data[mask]

    # Create a date series to be used in the clean dataframe
    date = pd.to_datetime(data['date'])

    # Create a temp series to be used in the clean dataframe
    temp = data['temp'].apply(float)

    # Cast the null value M to zero to enable create of the raw precipitation series cast to floats
    data.p01i[data.p01i == 'M'] = 0
    raw_precipitation = data['p01i'].apply(float)

    # Create a precipitation series to be used in the clean dataframe
    precipitation = raw_precipitation.apply(lambda x: True if (x > 0) else False)

    # Convert sky coverage data to clear or cloudy and create an overcast series to be used in the clean dataframe
    sky_elements = ['skyc1', 'skyc2', 'skyc3']
    data.skyc1 = data.skyc1.astype(str)
    data.skyc2 = data.skyc2.astype(str)
    data.skyc3 = data.skyc3.astype(str)
    sky_agg = data[sky_elements].values.tolist()
    sky_reduce = [['overcast' if (('BKN' in element) or ('OVC' in element) or ('VV' in element)) else 'clear'
                    for element in row] for row in sky_agg]
    overcast = pd.Series([True if 'overcast' in row else False for row in sky_reduce])
    overcast.index = date.index

    # Cast the null value 'M' to 10.00 to enable the creation of a poor visibility series
    data.vsby[data.vsby == 'M'] = 10.00
    raw_visibility = data['vsby'].apply(float)
    poor_visibility = pd.Series([True if value < 0.50 else False for value in raw_visibility])
    poor_visibility.index = date.index

    # Cast the null value 'M' to 0 to enable the creation of a windy series
    data.sknt[data.sknt == 'M'] = 0.00
    data.gust[data.gust == 'M'] = 0.00
    wind_speed = data['sknt'].apply(float)
    gust_speed = data['gust'].apply(float)
    wind_df = pd.concat([wind_speed, gust_speed], axis=1)
    # Finally apply the function f to enable the creation of the windy column
    windy = wind_df.apply(f, axis=1)

    """Create the cleaned dataframe by concatenating the date, temp, precipitation, overcast, poor_visibility
    and windy series"""
    df = pd.concat([date, temp, precipitation, overcast, poor_visibility, windy], axis=1)
    df.columns = ['date', 'temp', 'precipitation', 'overcast', 'poor_visibility', 'windy']
    cleaned_df = df[(df['date'] > '2006-12-31') & (df['date'] < '2018-04-03')]
    return cleaned_df

def f(row):
    """Function to be able to create the windy series with windy being true if wind speed is above 10 knots
    or gust speed is above 20 knots"""
    if row['sknt'] >= 10.00:
        val = True
    elif row['gust'] >= 20.00:
        val = True
    else:
        val = False
    return val




In [12]:
# %load ../src/pass_data_clean.py
import pandas as pd

def clean_pass_data(filename):
    """Take Snoqualmie pass closure data file and clean it ready for input to model.
    Input: xlsx file
    Output: Pandas dataframe
    """
    data = pd.read_excel(filename, header=[1])

    #drop unnamed/unnecessary columns
    data.drop(data.columns[[11,12,13,14]], axis=1, inplace=True)

    #drop unnecessary secondary incident columns
    data.drop(data.columns[[1,8]], axis=1, inplace=True)

    #rename 'Incident...' columns to start_time and end_time 
    data.rename(columns={'INCIDENT START TIMES FOR EACH DIRECTION':'start_time'}, inplace=True)
    data.rename(columns={'INCIDENT END TIMES - DIRECTIONAL':'end_time'}, inplace=True)

    #use only dates from 2007-01-01 to match with available weather and traffic volume data
    df = data[(data['start_time'] > '2006-12-31')]

    #rename 'Delay Time Total' to delay
    df.rename(columns={'Delay Time Total':'delay'}, inplace=True)

    #drop row with nan value in delay
    df = df.dropna(subset=['delay'])

    #create a westbound pandas series with True if westbound and false if eastbound
    westbound = pd.Series([True if value == 'WB' else False for value in df.DIRECTION])

    #create a snow pandas series with True if weather description contains sn, false otherwise
    snow = df.WEATHER.str.contains('sn', case=False, na=False, regex=True)

    #create pandas series for start and end times
    start_time = pd.to_datetime(df['start_time'])
    end_time = pd.to_datetime(df['end_time'])

    #ensure that all the pandas series created have the same index
    westbound.index = start_time.index
    snow.index = start_time.index
    end_time.index = start_time.index

    #create cleaned df with the series created
    cleaned_df = pd.concat([start_time, end_time, westbound, snow], axis=1)

    #rename columns
    cleaned_df.rename(columns={0:'westbound', 'WEATHER':'snow'}, inplace=True)

    return cleaned_df



In [17]:
# %load ../src/combine_data.py
import pandas as pd
#from pass_data_clean import clean_pass_data

pass_closure_df = clean_pass_data('Cumulative_Snoqualmie_Pass_Delay_Closures_1992_2018.xlsx')

def get_pass_closure(date_time):
    """take a date_time and check if it is between the start and end times of a closure event
    input: datetime
    output: boolean
    """
    start_end_times = list(zip(pass_closure_df.start_time, pass_closure_df.end_time))
    for row in start_end_times:
        if row[0] <= date_time <= row[1]:
            return True
    return False

def add_pass_closed(df):
    """take the weather df and add a new column for whether or not the pass is closed at each date_time
    input: pandas dataframe
    output: pandas dataframe
    """
    df['pass_closed'] = df['date'].map(get_pass_closure)
    return df

def true_false_to_one_zero(df):
    """take the combined df and change all the true/false values to 1/0
    input: pandas dataframe
    output: pandas dataframe
    """
    df[['precipitation', 'overcast', 'poor_visibility', 'windy', 'pass_closed']] = (
        df[['precipitation', 'overcast', 'poor_visibility', 'windy', 'pass_closed']] == True).astype(int)
    return df

def aggregate_data_to_daily(df):
    """take the combined df and aggregate the data into daily rather than hourly data to be used to train the model
    input: pandas dataframe
    output: pandas dataframe
    """
    df.index = df.date
    daily_df = df.resample("D").agg({'temp':'mean','precipitation':'max', 'overcast':'max', 'poor_visibility':'max', 'windy':'max', 'pass_closed':'max'})
    daily_df.dropna(inplace=True)
    daily_df.reset_index(inplace=True)
    daily_df.rename(columns={'index':'date'}, inplace=True)
    return daily_df


/Users/leanne/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [31]:
# %load ../src/model.py
import pandas as pd
import numpy as np
#from pipeline_classes import Featurizer
#from weather_data_clean import clean_weather_data
#from pass_data_clean import clean_pass_data
#from combine_data import get_pass_closure, add_pass_closed, true_false_to_one_zero, aggregate_data_to_daily
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import pickle

def get_data():
    """function to get all the original data that is required to train the model
    Output: pandas dataframe to use to train model"""
    weather_df = clean_weather_data('../data_exploration/ASOS_stampede_pass/SMP-2.txt')
    combined_df = add_pass_closed(weather_df)
    combined_df = true_false_to_one_zero(combined_df)
    daily_df = aggregate_data_to_daily(combined_df)
    return daily_df

def get_training_data():
    """get the training data that used to train the model
    Output: X, y used to train the model"""
    df = get_data()
    y = df['pass_closed']
    X = df.drop('pass_closed', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    return X_train, X_test, y_train, y_test

def pass_pipeline():
    """instantiate a pipeline object"""
    pipeline = Pipeline([
        ('featurizer', Featurizer()),
        ('model', RandomForestClassifier(n_estimators=600, 
                                         max_depth=40))
        ])
    return pipeline

def pickle_pipeline(pipeline, output_name):
    """Save fitted pipeline to pickle file"""
    with open(output_name, 'wb') as f:
        pickle.dump(pipeline, f)


In [32]:
X_train, X_test, y_train, y_test = get_training_data()

/Users/leanne/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [70]:
X_train

,date,temp,precipitation,overcast,poor_visibility,windy
1962,2014-06-11,57.368750,0.0,1.0,0.0,0.0
1839,2013-02-18,22.460000,0.0,1.0,0.0,0.0
655,2008-11-11,40.718261,1.0,1.0,1.0,0.0
610,2008-09-27,49.527500,0.0,1.0,1.0,0.0
2228,2016-06-04,67.925000,0.0,0.0,0.0,0.0
1444,2011-07-09,47.588000,0.0,0.0,0.0,0.0
1271,2010-10-26,30.380000,1.0,1.0,1.0,0.0
2209,2016-05-16,42.350000,0.0,1.0,1.0,0.0
1357,2011-02-21,23.747000,1.0,1.0,1.0,1.0
2031,2014-08-22,51.080000,0.0,0.0,0.0,0.0


In [78]:
features = Featurizer()

In [79]:
features.transform(X_train)

,temp,precipitation,overcast,poor_visibility,windy,year,month,day,dayofweek_Friday,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday,dayofweek_Wednesday
date,,,,,,,,,,,,,,,
2014-06-11,57.368750,0.0,1.0,0.0,0.0,2014,6,11,0,0,0,0,0,0,1
2013-02-18,22.460000,0.0,1.0,0.0,0.0,2013,2,18,0,1,0,0,0,0,0
2008-11-11,40.718261,1.0,1.0,1.0,0.0,2008,11,11,0,0,0,0,0,1,0
2008-09-27,49.527500,0.0,1.0,1.0,0.0,2008,9,27,0,0,1,0,0,0,0
2016-06-04,67.925000,0.0,0.0,0.0,0.0,2016,6,4,0,0,1,0,0,0,0
2011-07-09,47.588000,0.0,0.0,0.0,0.0,2011,7,9,0,0,1,0,0,0,0
2010-10-26,30.380000,1.0,1.0,1.0,0.0,2010,10,26,0,0,0,0,0,1,0
2016-05-16,42.350000,0.0,1.0,1.0,0.0,2016,5,16,0,1,0,0,0,0,0
2011-02-21,23.747000,1.0,1.0,1.0,1.0,2011,2,21,0,1,0,0,0,0,0


In [80]:
pipe = pass_pipeline()

In [81]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('featurizer', Featurizer(cols=['date', 'temp', 'precipitation', 'overcast', 'poor_visibility', 'windy'])), ('model', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrea...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [82]:
probs = pipe.predict_proba(X_test)

In [91]:
X_test

,date,temp,precipitation,overcast,poor_visibility,windy
443,2008-04-08,28.252727,1.0,1.0,1.0,0.0
2394,2016-11-20,33.686957,1.0,1.0,1.0,0.0
761,2009-05-07,34.160000,1.0,1.0,1.0,1.0
651,2008-11-07,38.870000,1.0,1.0,1.0,0.0
2674,2017-09-19,38.283333,1.0,1.0,1.0,0.0
1397,2011-05-22,36.888800,1.0,1.0,1.0,0.0
2766,2017-12-20,24.747826,1.0,1.0,1.0,0.0
1684,2012-07-29,51.620000,0.0,1.0,1.0,0.0
789,2009-06-04,68.502500,1.0,1.0,1.0,0.0
2661,2017-09-06,69.116667,0.0,1.0,0.0,0.0


In [83]:
probs

array([[0.93833333, 0.06166667],
       [0.93166667, 0.06833333],
       [0.965     , 0.035     ],
       ...,
       [0.965     , 0.035     ],
       [0.92833333, 0.07166667],
       [0.99666667, 0.00333333]])

In [84]:
probs[:,1].max()

0.815

In [85]:
probs[:,1].min()

0.0

In [86]:
probs[:,1].mean()

0.1075232288037166

In [88]:
# %load ../src/scrape_weather.py
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime

def get_raw_forecast(day):
    '''
    Get raw text from weather.com's 10 day forecast, for the day passed into the function.
    Day = 1 will be today.
    
    Input: int
    Day between 1 and 15, representing the day of the desired forecast.
    
    Output: list of strings
    Raw text for the day chosen, in the following order:
        [daymonth date, weather forecast, hi/lo temp, % chance precipitation, 
         wind speed and direction, humidity]
    '''
    page_link = 'https://weather.com/weather/tenday/l/USWA0413:1:US'
    page_response = requests.get(page_link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")

    web_predictions = []
    for i in range(0,105):
        predictions = page_content.find_all('td')[i].text
        web_predictions.append(predictions)
    return web_predictions[(day*7 - 6):(day*7)]

def get_raw_forecast_dataframe():
    ''' 
    Get latest raw weather forecast dataframe for the next 15 days.
    Output: raw dataframe.
    '''
    #Get list of forecasts for the next 15 days
    forecasts = []
    for day in range(1,16):
        forecasts.append(get_raw_forecast(day))
    
    #Set todays date to be used as the start of the dataframe index
    todays_date = datetime.datetime.now().date()
    index = pd.date_range(todays_date, periods=15, freq='D')
    #Set the columns for the dataframe
    columns = ['date','weather','temp', 'precipitation', 'wind', 'humidity']
    #Create the raw forecast dataframe with forecasts list, index and column headings
    forecasts_df = pd.DataFrame(forecasts, index=index, columns=columns)
    return forecasts_df

def get_hi_temperature(temp_string):
    '''
    Take the hi/lo string from weather.com and convert to an integer of the high temperature.
    Input: string
    Output: int
    '''
    numbers = [str(num) for num in range(0,10)]
    temp = ''
    for char in temp_string:
        if char in numbers:
            temp += char
        else:
            break
    #Add error handling for change in website that has -- for high temp for 'Tonight'
    if temp == '':
        return get_low_temperature(temp_string)
    return int(temp)

def get_low_temperature(temp_string):
    '''
    Take the hi/lo string from weather.com and convert to an integer of the
    low temperature.
    Input: string
    Output: int
    '''
    numbers = [str(num) for num in range(0,10)]
    temp = ''
    hilo = temp_string
    for char in reversed(hilo[:-1]):
        if char in numbers:
            temp = char + temp
        else:
            break
    return int(temp)

def get_wind_speed(temp_string):
    '''
    Take the wind string from weather.com and convert to an integer of the wind speed.
    Input: string
    Output: int
    '''
    numbers = [str(num) for num in range(0,10)]
    temp = ''
    for char in temp_string:
        if char in numbers:
            temp += char
    return int(temp)

def get_overcast(weather_string):
    """
    Take the weather string from weather.com and convert to a 1 if string contains words in overcast_list
    otherwise return a 0
    Input: string
    output: int(0 or 1)
    """
    overcast_list = ['Cloudy', 'Snow', 'Rain', 'Showers', 'Thunderstorms']
    for word in weather_string.split():
        if word in overcast_list:
            return 1
        else:
            return 0

def get_poor_visibility(weather_string):
    """
    Take the weather string from weather.com and convert to a 1 if string contains words in poor_visibility_list
    otherwise return a 0
    Input: string
    output: int(0 or 1)
    """
    poor_visibility_list = ['Snow', 'Rain', 'Fog', 'Mist']
    for word in weather_string.split():
        if word in poor_visibility_list:
            return 1
        else:
            return 0

def snoqualmie_pass_prediction_components():
    """
    Get latest cleaned features from weather.com to be used to gain closure probabilities
    Output: Clean dataframe to be used for predicting probability of closure
    """
    forecasts_df = get_raw_forecast_dataframe()
    
    #Extract high/low and calulate average temperature
    forecasts_df['high'] = forecasts_df['temp'].map(get_hi_temperature)
    forecasts_df['low'] = forecasts_df['temp'].map(get_low_temperature)
    forecasts_df['average'] = ((forecasts_df['high'] + forecasts_df['low']) / 2)

    #Re-use get_hi_temperature function to get just the integer from precipitation
    forecasts_df['precip'] = forecasts_df['precipitation'].map(get_hi_temperature)
    #Set precipitation to 1 if chance of precipitation is >= 30%
    forecasts_df['is_precipitating'] = forecasts_df['precip'].apply(lambda x: 1 if (x >= 30) else 0)

    #Extract wind speed and set to 1 if windspeed >= 10mph
    forecasts_df['wind_int'] = forecasts_df['wind'].map(get_wind_speed)
    forecasts_df['windy'] = forecasts_df['wind_int'].apply(lambda x: 1 if (x >= 10) else 0)

    #Infer overcast and poor visibility
    forecasts_df['overcast'] = forecasts_df['weather'].map(get_overcast)
    forecasts_df['poor_visibility'] = forecasts_df['weather'].map(get_poor_visibility)

    #Drop unwanted columns
    forecasts_df = forecasts_df.drop(['temp','high','low','precipitation','precip','wind','wind_int','weather','humidity','date'], axis=1)

    #Add date index to be new date column
    forecasts_df.reset_index(inplace=True)

    #Rename columns
    forecasts_df.rename(columns={'average':'temp', 'is_precipitating':'precipitation', 'index':'date' }, inplace=True)

    return forecasts_df



In [89]:
forecasts_df = snoqualmie_pass_prediction_components()

In [90]:
forecasts_df

,date,temp,precipitation,windy,overcast,poor_visibility
0,2018-12-04,22.0,0,0,0,0
1,2018-12-05,21.0,0,0,0,0
2,2018-12-06,19.5,0,0,0,0
3,2018-12-07,21.5,0,0,0,0
4,2018-12-08,24.5,0,0,0,0
5,2018-12-09,26.0,1,0,1,1
6,2018-12-10,27.0,1,0,1,1
7,2018-12-11,28.5,1,0,1,1
8,2018-12-12,27.5,1,0,1,1
9,2018-12-13,27.0,1,0,1,1


In [92]:
forecast_probs = pipe.predict_proba(forecasts_df)

In [93]:
forecast_probs

array([[0.94166667, 0.05833333],
       [0.91166667, 0.08833333],
       [0.935     , 0.065     ],
       [0.94333333, 0.05666667],
       [0.93166667, 0.06833333],
       [0.85      , 0.15      ],
       [0.85      , 0.15      ],
       [0.815     , 0.185     ],
       [0.88666667, 0.11333333],
       [0.88833333, 0.11166667],
       [0.85666667, 0.14333333],
       [0.92666667, 0.07333333],
       [0.905     , 0.095     ],
       [0.855     , 0.145     ],
       [0.74833333, 0.25166667]])

In [96]:
forecast_probs[:,1].round(2)

array([0.06, 0.09, 0.06, 0.06, 0.07, 0.15, 0.15, 0.18, 0.11, 0.11, 0.14,
       0.07, 0.1 , 0.14, 0.25])